# Chatbot Training

## Imports

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import re

Run the following if first time running nltk

In [63]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emcco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emcco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Corpus Processing

In [64]:
words=[]
class_names = []
classes = []
documents = []
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern.lower())
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in class_names:
            class_names.append(intent['tag'])
            classes.append((intent['tag'], intent['context']))

In [65]:
print(documents[:10])

[(['hi', 'there'], 'greeting'), (['how', 'are', 'you'], 'greeting'), (['hey'], 'greeting'), (['good', 'day'], 'greeting'), (['hi'], 'greeting'), (['hello'], 'greeting'), (['yo'], 'greeting'), (['bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye')]


In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [66]:
# Parse documents

words = [lemmatizer.lemmatize(w.lower()) for w in words if len(w) > 1]
words = sorted(list(set(words)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

34 documents
5 classes [('greeting', 'general'), ('goodbye', 'general'), ('thanks', 'general'), ('options', 'general'), ('connect_spotify', 'general')]
54 unique lemmatized words ["'s", 'appreciate', 'are', 'awesome', 'be', 'bye', 'call', 'can', 'connect', 'could', 'data', 'day', 'do', 'for', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'how', 'in', 'is', 'it', 'later', 'login', 'me', 'next', 'offered', 'open', 'option', 'out', 'peace', 'provide', 'pull', 'quit', 'see', 'sign', 'spotify', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'yo', 'you']


In [67]:
# initializing training data
training = []
class_row_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create sparse array to represent word vector
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # create sparse array to represent target class
    class_row = list(class_row_empty)
    class_row[class_names.index(doc[1])] = 1

    training.append([bag, class_row])

# shuffle features
random.shuffle(training)
training = np.array(training)

# create train list. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [68]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)
model.save(os.path.join(os.getcwd(),'chatbot_model.h5'), hist)

print("model created")

Epoch 1/50
7/7 [==============================] - 0s 567us/step - loss: 1.6241 - accuracy: 0.1471
Epoch 2/50
7/7 [==============================] - 0s 496us/step - loss: 1.5992 - accuracy: 0.2353
Epoch 3/50
7/7 [==============================] - 0s 709us/step - loss: 1.5505 - accuracy: 0.3824
Epoch 4/50
7/7 [==============================] - 0s 850us/step - loss: 1.5230 - accuracy: 0.3235
Epoch 5/50
7/7 [==============================] - 0s 709us/step - loss: 1.5371 - accuracy: 0.3235
Epoch 6/50
7/7 [==============================] - 0s 567us/step - loss: 1.4072 - accuracy: 0.5588
Epoch 7/50
7/7 [==============================] - 0s 638us/step - loss: 1.3899 - accuracy: 0.5294
Epoch 8/50
7/7 [==============================] - 0s 567us/step - loss: 1.3487 - accuracy: 0.5000
Epoch 9/50
7/7 [==============================] - 0s 638us/step - loss: 1.2947 - accuracy: 0.5294
Epoch 10/50
7/7 [==============================] - 0s 709us/step - loss: 1.2113 - accuracy: 0.6765
Epoch 11/50
7/7 [==